# Arctic

> Arctic helper scripts and functions.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# | default_exp arctic

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | export
import os

import click
from arcticdb import Arctic, LibraryOptions
import hydra
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf
from pathlib import Path
from lobster_tools.config import (
    MainConfig,
    Overrides,
    NASDAQExchange,
    ETFMembers,
    register_configs,
    get_config,
)
from lobster_tools.preprocessing import Data, Lobster, infer_ticker_to_date_range, infer_ticker_to_ticker_path, infer_ticker_dict
import sys
import pandas as pd
from logging import Logger
from datetime import date

When using the `@hydra.main` decorator, `register_configs` must be called. If simply using a notebook or writing the CLIs with `click`, it is enough to use `get_config`.

In [ ]:
# | export
register_configs()
cfg = get_config(overrides=Overrides.full_server)

In [ ]:
# | export
CONTEXT_SETTINGS = dict(
    help_option_names=["-h", "--help"],
    token_normalize_func=lambda x: x.lower() if isinstance(x, str) else x,
    show_default=True,
)

In [ ]:
# | export
def get_arctic_library(db_path, library):
    conn = f"lmdb://{db_path}"
    arctic = Arctic(conn)
    arctic_library = arctic[library]
    return arctic_library

In [ ]:
# | exports
# | code-fold: true
@click.command(context_settings=CONTEXT_SETTINGS)
@click.option("-d", "--db_path", default=cfg.db.db_path, help="database path")
@click.option("-l", "--library", default=cfg.db.library, help="library name")
def arctic_list_symbols(db_path, library) -> None:
    """List symbols in the arcticdb library."""
    arctic_library = get_arctic_library(db_path=db_path, library=library)
    print(f"Symbols in library {library}")
    print(arctic_library.list_symbols())

In [ ]:
# | exports
# | code-fold: true
@click.command(context_settings=CONTEXT_SETTINGS)
@click.option("-d", "--db_path", default=cfg.db.db_path, help="database path")
@click.option("-l", "--library", default=cfg.db.library, help="library name")
def arctic_create_new_library(db_path, library) -> None:
    """Create a blank new arcticdb library."""
    conn = f"lmdb://{db_path}"
    arctic = Arctic(conn)
    arctic.create_library(library) 
    print(arctic[library])

In [ ]:

# | exports
# | code-fold: true
@click.command(context_settings=CONTEXT_SETTINGS)
@click.option("-d", "--db_path", default=cfg.db.db_path, help="database path")
def arctic_list_libraries(db_path) -> None:
    """List arcticdb libraries"""

    conn = f"lmdb://{db_path}"
    arctic = Arctic(conn)
    print(arctic.list_libraries())

In [ ]:
# | exports
# | code-fold: true
@click.command(context_settings=CONTEXT_SETTINGS)
@click.option("-d", "--db_path", default=cfg.db.db_path, help="database path")
@click.option("-l", "--library", default=cfg.db.library, help="library name")
def arctic_delete_library(db_path, library) -> None:
    """Delete arcticdb library"""

    user_input = input("Proceed by deleting this entire library? (y/n): ")
    user_input = user_input.lower()
    match user_input:
        case "y":
            pass
        case "n":
            sys.exit(0)
        case _:
            sys.exit(1)

    conn = f"lmdb://{db_path}"
    arctic = Arctic(conn)
    arctic.delete_library(library) 

In [ ]:
# | exports
# | code-fold: true
@click.command(context_settings=CONTEXT_SETTINGS)
@click.option("-d", "--db_path", default=cfg.db.db_path, help="database path")
@click.option("-l", "--library", default=cfg.db.library, help="library name")
@click.option("-t", "--ticker", required=True, help="ticker to print")
@click.option("-s", "--start_date", default=None, help="start date")
@click.option("-e", "--end_date", default=None, help="end date")
def arctic_read_symbol(db_path, library, ticker, start_date, end_date,
):
    """Print df.head() and available columns for ticker in arcticdb library."""
    arctic_library = get_arctic_library(db_path=db_path, library=library)

    if start_date and end_date:
        start_datetime = pd.Timestamp(f"{start_date}T{NASDAQExchange.exchange_open}")
        end_datetime = pd.Timestamp(f"{end_date}T{NASDAQExchange.exchange_close}")
        date_range = (start_datetime, end_datetime)
        df = arctic_library.read(ticker, date_range=date_range).data
    else:
        df = arctic_library.read(ticker).data
    
    print(f"Printing df.head() and df.tail() for ticker {ticker}")
    print(df.head())
    print(df.tail())

Write to arctic again

In [ ]:
# | exports
# | code-fold: true
@click.command(context_settings=CONTEXT_SETTINGS)
@click.option(
    "-c", "--csv_path", default=cfg.data_config.csv_files_path, help="csv files path"
)
@click.option("-d", "--db_path", default=cfg.db.db_path, help="database path")
@click.option("-l", "--library", default=cfg.db.library, help="library name")
@click.option("-t", "--ticker", required=True, help="ticker to write to db")
@click.option("-s", "--start_date", default="2020-01-01", help="start date")
@click.option("-e", "--end_date", default="2020-02-01", help="end date")
def arctic_write_symbol(
    db_path,
    library,
    csv_path,
    ticker,
    start_date,
    end_date,
):
    arctic_library = get_arctic_library(db_path=db_path, library=library)

    # if ticker in arctic_library.list_symbols():
    #     print("warning - there is already data for ths ticker")
    #     user_input = input("Proceed by adding data to this symbol? (y/n): ")
    #     user_input = user_input.lower()
    #     match user_input:
    #         case "y":
    #             pass
    #         case "n":
    #             sys.exit(0)
    #         case _:
    #             sys.exit(1)

    date_range = (start_date, end_date)
    data = Data(
        directory_path=csv_path,
        ticker=ticker,
        date_range=date_range,
        aggregate_duplicates=False,
    )
    lobster = Lobster(data=data)
    df = pd.concat([lobster.messages, lobster.book], axis=1)

    arctic_library.write(symbol=ticker, data=df)

In [ ]:
# | exports
# | code-fold: true
@click.command(context_settings=CONTEXT_SETTINGS)
@click.option(
    "-c", "--csv_path", default=cfg.data_config.csv_files_path, help="csv files path"
)
@click.option("-d", "--db_path", default=cfg.db.db_path, help="database path")
@click.option("-l", "--library", default=cfg.db.library, help="library name")
@click.option("-s", "--start_date", default=None, help="start date")
@click.option("-e", "--end_date", default=None, help="end date")
def arctic_generate_jobs(csv_path, db_path, library, start_date, end_date):
    ticker_date_dict = infer_ticker_to_date_range(csv_path)
    with open('arctic_commands.txt', 'w') as f:
        for ticker, (inferred_start_date, inferred_end_date) in ticker_date_dict.items():
            # if date is None use the inferred date, otherwise use the CLI argument
            start_date = start_date or inferred_start_date
            end_date = end_date or inferred_end_date
            f.write(f"arctic_write_symbol --csv_path={csv_path} --db_path={db_path} --library={library} --ticker={ticker} --start_date={start_date} --end_date={end_date} \n")

In [ ]:
# | exports
# | code-fold: true
@click.command(context_settings=CONTEXT_SETTINGS)
@click.option(
    "-z",
    "--zip_path",
    default="/nfs/lobster_data/lobster_raw/2016",
    help="zip files path",
)
@click.option(
    "-c", "--csv_path", default=cfg.data_config.csv_files_path, help="csv files path"
)
@click.option(
    "-e", "--etf", default=None, help="restrict to subset specified by ETF members"
)
def zip_generate_jobs(zip_path, csv_path, etf):
    # ticker_date_dict = infer_ticker_to_ticker_path(zip_path)
    # print(ticker_date_dict)
    # if etf:
    #     print(ETFMembers().mapping[etf])
    #     ticker_date_dict = {
    #         ticker: ticker_path
    #         for ticker, ticker_path in ticker_date_dict.items()
    #         if ticker in ETFMembers().mapping[etf] + [etf]
    #     }
    # print(ticker_date_dict)
    ticker_dict = infer_ticker_dict(zip_path)
    with open("zip_commands.txt", "w") as f:
        for ticker, dict_ in ticker_dict.items():
            full = dict_["full"]
            ticker_till_end = dict_["ticker_till_end"]
            f.write(f"mkdir {csv_path}/{ticker_till_end}\n")
            f.write(f"/nfs/home/nicolasp/usr/bin/7z x {full} -o{ticker_till_end}\n")

In [ ]:
# | exports
# | code-fold: true
@click.command(context_settings=CONTEXT_SETTINGS)
@click.option(
    "-c", "--csv_path", default=cfg.data_config.csv_files_path, help="csv files path"
)
@click.option("-d", "--db_path", default=cfg.db.db_path, help="database path")
@click.option("-l", "--library", default=cfg.db.library, help="library name")
@click.option("-t", "--ticker", required=True, help="ticker to write to db")
@click.option("-s", "--start_date", default="2020-01-01", help="start date")
@click.option("-e", "--end_date", default="2020-02-01", help="end date")
def arctic_dump_all(
    db_path,
    library,
    csv_path,
    ticker,
    start_date,
    end_date,
):
    arctic_library = get_arctic_library(db_path=db_path, library=library)

    if ticker in arctic_library.list_symbols():
        print("warning - there is already data for ths ticker")
        user_input = input("Proceed by adding data to this symbol? (y/n): ")
        user_input = user_input.lower()
        match user_input:
            case "y":
                pass
            case "n":
                sys.exit(0)
            case _:
                sys.exit(1)

    date_range = (start_date, end_date)
    data = Data(
        directory_path=csv_path,
        ticker=ticker,
        date_range=date_range,
        aggregate_duplicates=False,
    )
    lobster = Lobster(data=data)
    df = pd.concat([lobster.messages, lobster.book], axis=1)
    print(df)

    arctic_library.append(symbol=ticker, data=df)

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()